In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
from collections import deque
memorySize=1000
lr=0.001
df=0.9
syncRate=10
miniBatchSize=32

In [2]:
# Define the model
def build_model():
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=(16,)))    # Input layer with 16 nodes
    model.add(layers.Dense(16, activation='relu'))     # Hidden layer with 16 nodes and ReLU activation
    model.add(layers.Dense(4, activation='softmax'))   # Output layer with 4 nodes and Softmax activation
    model.compile(optimizer='adam',
                  loss='mse',lr=lr)
    return model


In [8]:
def stateToDQNInput(state,num_states=16):
    one_hot_vector = np.zeros(num_states)
    one_hot_vector[state] = 1
    return one_hot_vector

In [10]:
import gym
env=gym.make("FrozenLake-v1", map_name="4x4", is_slippery=False, render_mode="rgb_array")
state=env.reset()
state

(0, {'prob': 1})

In [12]:
stateToDQNInput(state[0])

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [13]:
env.action_space.sample()

3

In [14]:
env.step(3)

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


(0, 0.0, False, False, {'prob': 1.0})

In [19]:
np.array([2,33,1,1]).argmax()

1

In [ ]:
def optimize(miniBatch,policyDQN,targetDQN):
    numStates=16
    currentQList=[]
    targerQList=[]
    for state,action,newState,reward,terminated in miniBatch:
        if terminated:
            target=reward
        else:
            target=


In [7]:
import gym
import random
def train(episodes,render=False, is_slippery=False):
    env=gym.make("FrozenLake-v1", map_name="4x4", is_slippery=False, render_mode="rgb_array")
    num_states=env.observation_space.n
    num_actions=env.action_space.n
    epsilon=1
    replayMemory=deque(maxlen=memorySize)
    policyDQN=build_model()
    targetDQN=build_model()
    targetDQN.set_weights(policyDQN.get_weights())
    stepCount=0
    rewardsPerEpisodes=np.zeros(episodes)
    for i in range(episodes):
        state=env.reset()[0]
        terminated=False
        truncated=False
        while(not truncated and not terminated):
            random_num = random.uniform(0, 1)
            if random_num > epsilon:
                action =policyDQN(stateToDQNInput(state)).argmax().item()
            else:
                action = env.action_space.sample()
            newState,reward,terminated,truncated,_=env.step(action)
            replayMemory.append((state,action,newState,reward,terminated))
            state=newState
            stepCount=stepCount+1
        if reward== 1:
            rewardsPerEpisodes[i]=1
        if(len(replayMemory)>miniBatchSize and np.sum(rewardsPerEpisodes)>=1):
            miniBatch=replayMemory.sample(miniBatchSize)
            optimize(miniBatch,policyDQN,targetDQN)